In [1]:
from langchain_experimental.agents.agent_toolkits.pandas.base import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai import ChatGoogleGenerativeAI
import matplotlib.pyplot as plt
import pandas as pd
from dotenv import load_dotenv
import os
import time
from google.api_core.exceptions import ResourceExhausted


# load environment varibles, bao loi neu khong co Key
load_dotenv()
google_api_key = os.environ.get("GOOGLE_API_KEY")
if not google_api_key:
    raise ValueError("Google API Key not found in environment variables.")

openai_api_key = os.environ.get("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("OpenAI API Key not found in environment variables.")


In [ ]:
import sys
sys.path.append("/Users/tuan.vo/llm-ai/dagpt") #THEM PATH VAO DE LOAD DUOC SCR

from scr.models.llms import load_llm

#   LUA CHON MODEL DE CHAY
#MODEL_NAME = "gpt-4o-mini"
#MODEL_NAME = "gemini-pro"
model_name = "gemini-1.5-pro-002"
api_key = google_api_key
try:
    llm = load_llm(api_key, model_name)
    print(llm)
except TypeError as e:
    print(f"Error: {e}")
except ValueError as e:
    print(f"Error: {e}")

#llm

In [ ]:
# DOC DU LIEU
df = pd.read_csv("../data/raw/updated_stock_data.csv")

# GOI HAM AGENT create_pandas_dataframe_agent
try:
    da_agent = create_pandas_dataframe_agent(
        llm=llm,
        df=df,
        agent_type='zero-shot-react-description',
        allow_dangerous_code=True,
        verbose=True,
        return_intermediate_steps=True,
)
except Exception as e:
    raise RuntimeError(f"Failed to create pandas DataFrame agent: {e}")
da_agent

In [4]:
# THEM HAM NAY DE BAO LOI ResourceExhausted 
def call_once(da_agent, query,):
    try:
        response = da_agent.invoke(query)
        return response
    except ResourceExhausted as e:
        print("Resource exhausted, cannot complete the request at this time.")
        raise
    except Exception as e:
        print(f"An error occurred: {e}")
        raise

def call_with_retry(da_agent, query, retries=2):
    for i in range(retries):
        try:
            response = da_agent.invoke(query)
            return response
        except ResourceExhausted as e:
            wait_time = 5 ** i  # Exponential backoff
            print(f"Resource exhausted, retrying in {wait_time} seconds...")
            time.sleep(wait_time)
        except Exception as e:
            print(f"An error occurred: {e}")
            raise
    raise RuntimeError("Maximum retry limit reached")

In [ ]:
query = "describe this data"
try:
    response = call_once(da_agent, query)
    print(response)
except RuntimeError as e:
    print(f"Failed to get a response: {e}")
#response = da_agent.invoke(query)
response

In [ ]:
# Access the input
input_query = response.get("input", "No input found.")
print(f"Input Query: {input_query}")
    
    # Access the output
output_result = response.get("output", "No output found.")
print(f"Output Result: {output_result}")
    
# Accessing intermediate_steps
intermediate_steps = response.get("intermediate_steps", [])
if intermediate_steps:
    # First element is an AgentAction object, so let's print its log
    agent_action, action_input = intermediate_steps[0]  # Unpack the tuple
    print("First intermediate step action:", agent_action)
    print("First intermediate step action input:", action_input)  # This is the tool input
    # Extract tool_input from agent_action
    # Assuming agent_action is structured with tool_input as a string
    tool_input_start = agent_action.tool_input.find('```python') + len('```python\n')
    tool_input_end = agent_action.tool_input.rfind('```')
    code = agent_action.tool_input[tool_input_start:tool_input_end].strip()  # Extracting the code between the markers
        
    print("Code extracted from tool_input:")
    print(code)    
    
    # Check if there's a second element in intermediate_steps
    if len(intermediate_steps) > 1:
        print("Second intermediate step output:", intermediate_steps[1])
    else:
        print("No second intermediate step output available.")    

else:
    print("No intermediate steps found.")

In [ ]:
code_reg = response["intermediate_steps"][-1][0].tool_input
def extract_python_code(data):
    # Loại bỏ phần đầu và cuối chứa ```python và ```
    data = data.replace("```python\n", "").replace("\n```", "")
    
    return data
python_code = extract_python_code(code_reg)
print(python_code)

In [ ]:
response['output']

In [ ]:
# print(response["intermediate_steps"][-1][0].tool_input['query'])